# Analysis of the molecular dynamics of CYP2D6

The `traj_analysis.py` is cutomized set of classes and functions to do the analysis of the MD trjectories and plot the figures. It needs to be in the same directory of the notebook. 

In [2]:
%run traj_analysis.py


## Loading the data 
`TRAJHOME` contains subdirectories that contain the trajectories and topologies for each of the analysed variants. 
We used the crytal structure of CYP2D6 (3tbg) as reference for the calculation `common_ref`  

In [3]:
import pytraj as pt
import pandas as pd 
import mdtraj as md
import numpy as np
import glob
import matplotlib.pylab as plt 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pytraj as pt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib.cm as cm
from scipy.spatial import distance

###########################################
#   Set these before running the notebook
###########################################
OUTPUTHOME="./CYP2D6_traj_analysis/"
# oath to topology and traj folders and files
TRAJHOME="/home/bsitabule/lustre/MD_simulations"
#TRAJHOME="/mnt/lustre/users/bsitabule/MD_simulations"
REFHOME="/home/bsitabule/lustre/MD_simulations/MD_analysis/CYP2D6_ref/new_CYP2D6.pdb"
#REFHOME ="/mnt/lustre/users/bsitabule/MD_analysis/CYP2D6_ref/new_CYP2D6.pdb"

common_ref = md.load(REFHOME)
start=1  
# subset of traj

# create a subdirectory for PCA within analysis
PCAHOME=OUTPUTHOME+"CYP2D6_data/PCA_test"
DATAHOME=OUTPUTHOME+"CYP2D6_data/"

import os
if  not os.path.exists(OUTPUTHOME) : # create container folder for output  
    os.mkdir(OUTPUTHOME)
if  not os.path.exists(OUTPUTHOME+"/CYP2D6_data/") : # create container for data folder
    os.mkdir(OUTPUTHOME+"/CYP2D6_data/")
if  not os.path.exists(OUTPUTHOME+"/figures/") : # create container for figures folder
    os.mkdir(OUTPUTHOME+"/figures/")

In [4]:
# path to traj files 
WT = TRAJHOME+"/CYP2D6_wt/CYP2D6_wt_500ns.nc"
l91m = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_L91M/Production_L91M_md/CYP2D6_L91M_500ns.nc"
p34s = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_P34S/Production_P34S_md/CYP2D6_P34S_500ns.nc"
s486t = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_S486T/Production_S486T_md/CYP2D6_S486T_500ns.nc"
y355c = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_Y355C/Production_Y355C_md/CYP2D6_Y355C_500ns.nc"
v338m = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V338M/CYP2D6_V338M_500ns.nc"
v104m = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V104M/CYP2D6_V104M_500ns.nc"
v136i = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V136I/CYP2D6_V136I_500ns.nc"
p267h = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_P267H/CYP2D6_P267H_500ns.nc"
r365h = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_R365H/CYP2D6_R365H_500ns.nc"
t107i = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_T107I/CYP2D6_T107I_500ns.nc"
v119l = TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V119L/CYP2D6_V119L_500ns.nc"

# path to topology files 
TOPwt=TRAJHOME+"/CYP2D6_wt/stripped.CYP2D6_wt_solvated.prmtop" 
TOPL91M=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_L91M/Production_L91M_md/stripped.CYP2D6_L91M_solvated.prmtop"
TOPP34S=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_P34S/Production_P34S_md/stripped.CYP2D6_P34S_solvated.prmtop"
TOPS486T=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_S486T/Production_S486T_md/stripped.CYP2D6_S486T_solvated.prmtop"
TOPY355C=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_Y355C/Production_Y355C_md/stripped.CYP2D6_Y355C_solvated.prmtop"
TOPV338M=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V338M/stripped.CYP2D6_V338M_solvated.prmtop"
TOPV104M=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V104M/stripped.CYP2D6_V104M_solvated.prmtop"
TOPV136I=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V136I/stripped.CYP2D6_V136I_solvated.prmtop"
TOPP267H=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_P267H/stripped.CYP2D6_P267H_solvated.prmtop"
TOPR365H=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_R365H/stripped.CYP2D6_R365H_solvated.prmtop"
TOPT107I=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_T107I/stripped.CYP2D6_T107I_solvated.prmtop"
TOPV119L=TRAJHOME+"/CYP2D6_missense_variants/CYP2D6_V119L/stripped.CYP2D6_V119L_solvated.prmtop"

traj_paths = [WT, l91m, p34s, s486t, y355c, v338m, v104m, v136i, p267h, r365h, t107i, v119l]
top_files = [ TOPwt, TOPL91M, TOPP34S,  TOPS486T, TOPY355C, TOPV338M, TOPV104M, TOPV136I, TOPP267H, TOPR365H, TOPT107I, TOPV119L]
labels = ["wt", "L91M","P34S", "S486T", "Y355C", "V338M", "V104M", "V136I", "P267H", "R365H", "T107I", "V119L"]

# RMSD and RMSF calculations


In [4]:
traj_wt = md.load(WT, top=TOPwt)
for trajectory,topology,label in  zip(traj_paths, top_files, labels  ) :
    mytraj = md.load(trajectory, top=topology)
    rmsd = RMSD(mytraj, reference=common_ref)
    rmsd.to_csv(OUTPUTHOME+"/CYP2D6_data/"+label+"_RMSD.csv", index=False)
    del rmsd
    wt_and_vars_trajs = [traj_wt, mytraj]
    rmsf = RmsfCalculation(wt_and_vars_trajs, common_ref, start=start, offset = 5, labels = ["Ref", label] )
    rmsf.CalculateRmsf()
    rmsf.dumpToCsv(output=OUTPUTHOME+"/CYP2D6_data/"+label+"_RMSF.csv")
    del rmsf

[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7350 7370 7371]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7349 7369 7370]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7355 7375 7376]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7342 7362 7363]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7353 7373 7374]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7353 7373 7374]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7355 7375 7376]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7355 7375 7376]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7345 7365 7366]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7357 7377 7378]
[   0    4   24 ... 7352 7372 7373]
[   0    4   24 ... 7355 7375 7376]
[   0    4   24 ... 7352 7372 7373]


# Essential dynamics data generation
Essential dynamics is conducted in pytraj so we have to re-read the trajectories again. but in an iterative way to avoid the memory overflow. 


### PCA
Running PCA analysis by calculating 20 modes (n_vecs=20) using backbone atoms.

In [5]:
tt=[]
len(tt)

0

In [6]:
# generating the DCCMs in text file

try:
    os.mkdir(PCAHOME)
except: 
    pass 
reference_crsytal = pt.load(REFHOME)

for traj, label, top in zip(traj_paths, labels, top_files): 
    dir4traj = PCAHOME+"/"+label
    try: 
        os.mkdir(dir4traj)
    except: 
        pass # the directory is already created
    read = pt.load(traj, top)
    pca_calculations = pt.pca(read[start:-1], ref=reference_crsytal,  mask='@CA,C,N', n_vecs=20)
    pcs = pca_calculations[0]
    np.savetxt( dir4traj+'/'+label+"_pca.txt", pcs, fmt="%.3f")
    eigenvalue = pca_calculations[1][0]
    np.savetxt( dir4traj+'/'+label+"_eigenvalues.txt", eigenvalue, fmt="%.5f")
    eigenvectors = pca_calculations[1][1]
    np.savetxt( dir4traj+'/'+label+"_eigenvectors.txt", eigenvectors, fmt="%.5f")
    del read

TypeError: '<' not supported between instances of 'list' and 'int'